In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import bz2
import json
from tld import get_tld
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
import math
from collections import Counter
from operator import itemgetter
import re
import pickle

nltk.download()
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from Data_clean_functions import *

### CREATE PATHS: change according to local file

In [ ]:
DATA_PATH = './data/'
FILE2019 = DATA_PATH + 'quotes-2019.json.bz2'
PATH_OUT = DATA_PATH + 'clean-quotes-2019.json.bz2'

In [ ]:
df_base = pd.read_json(FILE2019, lines=True, compression='bz2', nrows=1000)
df_base

## Create a dictionnary of categories and associated synonyms

In [ ]:
matchers = {"art": ["art", "paint", "draw", "museum"], \
            "business": ["business", "finance", "economy", "commerce", "bank", "money", "trade"], \
            "entertainment":["entertainment"], 
            "fashion":["fashion", "couture", "designer"], \
            "medicine":["medicine", "health", "pharmacy", "wellbeing", "body"], \
            "music":["music", "song", "album", "concert"], \
            "politics":["politics", "government"], \
            "science":["science", "research"], \
            "sport": ["sport", "football", "athletics", "swimming", "rugby", "tennis", "volleyball", "ski"]}

In [ ]:
# Don't run this as doesn't return good list

'''

def find_synonyms(matchers):
    for category in matchers:
        synonyms = []
        for i in range(len(matchers[category])):
            word = matchers[category][i]
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    synonyms.append(l.name())
            matchers[category] = synonyms
    return matchers

matchers = find_synonyms(matchers)

'''

In [ ]:
generalizeDictionary(matchers)

In [ ]:
matchers

### PROCESSING DATA

In [ ]:
# Pickle the chunks of dataframe for later
n_chunks = 0

with pd.read_json(FILE2019, lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        n_chunks += 1
        chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers)
        with open(PATH_OUT, 'ab') as d_file:
            pickle.dump(chunk_cleaned, d_file)       

In [ ]:
# Open pickled file
for chunk_nbr in range(n_chunks):
    with open(PATH_OUT, 'rb') as d_file:
        chunk = pickle.load(d_file)
        # Need to load m times if you want to get to the m^th file

### Without pickle / writing to new json

In [ ]:
with pd.read_json(FILE2019, lines=True, compression='bz2', chunksize=1000) as df_reader:
    with bz2.open(PATH_OUT, 'wb') as d_file:
        for chunk in df_reader:
            chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers)
            chunk_json = chunk_cleaned.to_json(orient='columns')#, index=False) #write it to pickle file instead
            d_file.write((chunk_json+'\n').encode('utf-8'))

In [ ]:
df = pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', orient='columns', compression='bz2')
df

In [ ]:
df = pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', orient='columns', compression='bz2', lines=True, nrows=100)
df

In [ ]:
with pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    
